In [105]:
import pandas as pd
import numpy
from tqdm import tqdm
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

df_accuracy = pd.read_csv('top_and_bottom_anime.csv')
df_accuracy = df_accuracy.drop(columns='Id')
df_accuracy = df_accuracy.drop(columns='Rating')
df_accuracy = df_accuracy.drop(columns='Demographic')
df_accuracy = df_accuracy.drop(columns='Genre')

new = pd.DataFrame()
le = preprocessing.LabelEncoder()
for i in range(df_accuracy.shape[1]):
    temp = df_accuracy.iloc[:,i].values.tolist()
    temp = le.fit_transform(temp)
    new[df_accuracy.columns[i]] = temp

df_true = pd.read_csv('anime_matrix_true.csv')
rows = df_true[['Id']].copy()


In [106]:
#this sets up df_predictions with the same rows as this train/test split
x = new
y = rows
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2, random_state=42)

#create cross training split for building the models
X_cross_train, X_cross_test, y_cross_train, y_cross_test = train_test_split(x_train, y_train, test_size=0.2, random_state=420)
df_predictions = y_cross_test.copy()

In [107]:
columns = df_true.columns[1:]
predictions = []

avg = []
for i in tqdm(range(len(columns))):
    x = new
    y = df_true[columns[i]]
    x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2, random_state=42)
    
    #create cross training split for building the models
    X_cross_train, X_cross_test, y_cross_train, y_cross_test = train_test_split(x_train, y_train, test_size=0.2, random_state=420)

    adaboost = AdaBoostClassifier(n_estimators=100, random_state=42)
    adaboost.fit(X_cross_train,y_cross_train)
    avg.append(adaboost.score(X_cross_test,y_cross_test))
    predictions.append(list(adaboost.predict(X_cross_test)))

# result

100%|██████████| 9028/9028 [13:03<00:00, 11.52it/s]


In [108]:
import pickle

file = open('predictions.p', 'wb')
pickle.dump(predictions, file)

In [109]:
file = open('predictions.p', 'rb')
predictions = pickle.load(file)

In [110]:
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
for i in tqdm(range(len(columns))):
    df_predictions[columns[i]] = predictions[i]

outputdf = df_predictions.copy()
outputdf.to_csv(f'predictions_matrix_adaboost_drop_genres.csv', index=False)

100%|██████████| 9028/9028 [00:06<00:00, 1291.30it/s]


In [111]:
import pickle

# open a file, where you ant to store the data
file = open('adaboost.model', 'wb')

# dump information to that file
pickle.dump(adaboost, file)

# close the file
file.close()

In [112]:
#creates true_values for only the shows included in this train/test split

df_true = pd.read_csv('anime_matrix_true.csv')
values = df_true.values
anime_ids = list(df_predictions['Id'])
true_values = []
for anime in anime_ids:
    for j in range(len(values)):
        if anime in values[j]:
            true_values.append(values[j])

true_values = numpy.array([numpy.array(xi) for xi in true_values])

In [113]:
#caluclate accuracy
value = 0
prediction_matrix = pd.read_csv('predictions_matrix_adaboost_drop_genres.csv')
pred_values = prediction_matrix.values

num_row = len(pred_values)
num_col = len(pred_values[0])
for i in tqdm(range(num_row)):
    for j in range(1,num_col):          #first item is the anime id, so we skip it
        if pred_values[i,j] == true_values[i,j]:
            value += 1

#subtracting 1 from column length b/c the first item is the anime id
percent = (value / ((num_row) * (num_col-1))) * 100
total = (num_row) * (num_col-1)
print(f'Correct predictions: {value}')
print(f'Total predictions: {total}')
print(f'Percent correct: {percent: .2f}%')

100%|██████████| 89/89 [00:00<00:00, 511.03it/s]

Correct predictions: 665250
Total predictions: 803492
Percent correct:  82.79%
